In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J. M.
Editora: CENGAGE LEARNING

Cap. 8: Heterocedasticidade (Heteroskedasticity)
Exemplo 8.3: Estatística LM robusta em relação à heterocedasticidade
             (HETEROSKEDASTICITY-ROBUST LM STATISTIC)
             
Arquivo com os dados: crime1.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import statsmodels.iolib.summary2 as sis

In [2]:
df = pd.read_excel('crime1.xls',
                   header=None,
                   usecols=[0, 3, 4, 6, 7, 8, 10, 11],
                   names=['narr86', 'pcnv', 'avgsen', 'ptime86', 'qemp86', 'inc86', 'black', 'hispan'])

In [3]:
df.head(2)

,narr86,pcnv,avgsen,ptime86,qemp86,inc86,black,hispan
0,0,0.38,17.6,12,0.0,0.0,0,0
1,2,0.44,0.0,0,1.0,0.8,0,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2725 entries, 0 to 2724
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   narr86   2725 non-null   int64  
 1   pcnv     2725 non-null   float64
 2   avgsen   2725 non-null   float64
 3   ptime86  2725 non-null   int64  
 4   qemp86   2725 non-null   float64
 5   inc86    2725 non-null   float64
 6   black    2725 non-null   int64  
 7   hispan   2725 non-null   int64  
dtypes: float64(4), int64(4)
memory usage: 170.4 KB


In [5]:
# Verifica se alguma variável possui dados faltantes
df.isna().sum()

narr86     0
pcnv       0
avgsen     0
ptime86    0
qemp86     0
inc86      0
black      0
hispan     0
dtype: int64

### Solução sugerida

In [6]:
model = smf.ols('narr86 ~ pcnv + avgsen + np.square(avgsen) + ptime86 + qemp86 + inc86 + black + hispan', data=df)
reg = model.fit()
reg_robust = model.fit(cov_type='HC0')    # implementa procedimento robusto em relação à heterocedasticidade

In [7]:
results_table = sis.summary_col(results=[reg, reg_robust],
                            float_format='%0.4f',
                            stars = False,
                            model_names=['se_usual',
                                         'se_robusto'])

results_table.add_title('Exemplo 8.3')
results_table.add_text(f'Núm. de obs.: {int(model.nobs)}')
print(results_table)

             Exemplo 8.3
                  se_usual se_robusto
-------------------------------------
Intercept         0.5670   0.5670    
                  (0.0361) (0.0402)  
pcnv              -0.1356  -0.1356   
                  (0.0404) (0.0336)  
avgsen            0.0178   0.0178    
                  (0.0097) (0.0101)  
np.square(avgsen) -0.0005  -0.0005   
                  (0.0003) (0.0002)  
ptime86           -0.0394  -0.0394   
                  (0.0087) (0.0062)  
qemp86            -0.0505  -0.0505   
                  (0.0144) (0.0142)  
inc86             -0.0015  -0.0015   
                  (0.0003) (0.0002)  
black             0.3246   0.3246    
                  (0.0454) (0.0584)  
hispan            0.1934   0.1934    
                  (0.0397) (0.0402)  
R-squared         0.0728   0.0728    
R-squared Adj.    0.0701   0.0701    
Standard errors in parentheses.
Núm. de obs.: 2725


In [8]:
# Estatística t usual
reg.tvalues

Intercept            15.725311
pcnv                 -3.358825
avgsen                1.840039
np.square(avgsen)    -1.738336
ptime86              -4.527519
qemp86               -3.499055
inc86                -4.345149
black                 7.146872
hispan                4.870607
dtype: float64

In [9]:
# Estatísticas t robusta
reg_robust.tvalues

Intercept            14.101635
pcnv                 -4.039638
avgsen                1.765289
np.square(avgsen)    -2.490154
ptime86              -6.334825
qemp86               -3.562351
inc86                -6.457633
black                 5.556660
hispan                4.806659
dtype: float64

In [11]:
'''
Estatística LM usual

H0: B_avgsen = B_avgsen2 = 0             # avgsen2: avgsen ao quadrado 
H1: pelo menos um dos parâmetros na hipótese nula é diferente de zero
'''
# Modelo restrito: sem "avgsen" e "avgsen2" 
model_restrito = smf.ols('narr86 ~ pcnv + ptime86 + qemp86 + inc86 + black + hispan', data=df)
reg_restrito = model_restrito.fit()

lm = reg.compare_lm_test(reg_restrito)   # Note que "reg" refere-se ao modelo irrestrito (com todas as variáveis)
print(f"estatística LM usual: {round(lm[0], 2)}")
print(f"valor p: {round(lm[1], 2)}")


estatística LM usual: 3.46
valor p: 0.18


In [14]:
'''
Estatística LM robusta em relação à heterocedasticidade

H0: B_avgsen = B_avgsen2 = 0             # avgsen2: avgsen ao quadrado 
H1: pelo menos um dos parâmetros na hipótese nula é diferente de zero
'''
# Modelo restrito: sem "avgsen" e "avgsen2" 
model_restrito = smf.ols('narr86 ~ pcnv + ptime86 + qemp86 + inc86 + black + hispan', data=df)
reg_restrito = model_restrito.fit()

lm_robusto = reg_robust.compare_lm_test(reg_restrito)
print(f"estatística LM robusta: {round(lm_robusto[0], 3)}")
print(f"valor p: {round(lm_robusto[1], 3)}")

estatística LM robusta: 4.02
valor p: 0.134


### Estatística LM robusta em relação à heterocedasticidade
Conforme passos estabelecidos no item 8-2a do livro do Wooldrige

In [ ]:
'''
Estatística LM robusta em relação à heterocedasticidade

H0: B_avgsen = B_avgsen2 = 0             # avgsen2: avgsen ao quadrado 
H1: pelo menos um dos parâmetros na hipótese nula é diferente de zero
'''

In [13]:
import scipy.stats

In [15]:
# Passo 1
model_restrito = smf.ols('narr86 ~ pcnv + ptime86 + qemp86 + inc86 + black + hispan', data=df)
reg_restrito = model_restrito.fit()
resid_model_restrito = reg_restrito.resid

In [16]:
# Passo 2
q = 2 # Número de variáveis independentes excluídas no modelo restrito, conforme Hipótese nula (avgsen e np.square(avgsen))

modelo_1 = smf.ols('avgsen ~ pcnv + ptime86 + qemp86 + inc86 + black + hispan', data=df)
reg_1 = modelo_1.fit()
residuos_1 = reg_1.resid 

modelo_2 = smf.ols('np.square(avgsen) ~ pcnv + ptime86 + qemp86 + inc86 + black + hispan', data=df)
reg_2 = modelo_2.fit()
residuos_2 = reg_2.resid

In [17]:
# Passo 3
produto_1 = residuos_1 * resid_model_restrito
produto_2 = residuos_2 * resid_model_restrito

df_apoio = pd.DataFrame({'produto_1': produto_1, 'produto_2': produto_2}) 

In [18]:
# Passo 4
modelo_final = smf.ols('1 ~ produto_1 + produto_2 - 1', data=df_apoio)   # - 1: retira o intercepto
reg_final = modelo_final.fit()

n = reg_final.nobs
SQR1 = reg_final.ssr         # Soma dos quadrados dos resíduos

estatistica_LM_robusta = n - SQR1
estatistica_LM_robusta

3.9970849092837852

In [19]:
valor_p = 1 - scipy.stats.chi2.cdf(estatistica_LM_robusta, df=q) # q é o número de variáveis excluídas conforme Passo 2
valor_p

0.13553268437609245

### Ponto de reversão de 'avgsen' sobre 'narr86'

In [20]:
import sympy as sp

In [21]:
'''
Ponto de reversão de 'avgsen' sobre 'narr86' (em que o efeito de 'avgsen' sobre 'narr86' passa de positivo para negativo)
Para encontrar o ponto de reversão devemos fazer: 
(d_narr86 / d_avgsen) = 0
Em que (d_narr86 / d_avgsen) é a derivada parcial de narr86 em relação à avgsen 
Note que o atributo "params" nos dá as estimativas de MQO
'''
reg.params

Intercept            0.567013
pcnv                -0.135595
avgsen               0.017841
np.square(avgsen)   -0.000516
ptime86             -0.039360
qemp86              -0.050507
inc86               -0.001480
black                0.324602
hispan               0.193380
dtype: float64

In [22]:
# Usamos o Sympy para calcular o ponto de reversão

# Definimos as variáveis
narr86, pcnv, avgsen, ptime86, qemp86, inc86, black, hispan =\
sp.symbols('narr86 pcnv avgsen ptime86 qemp86 inc86 black hispan')

# Definimos "narr86" como sendo uma função
narr86 = sp.symbols('narr86', cls=sp.Function)

# Definimos a função narr86
narr86 = reg.params[0] + reg.params[1]*pcnv + reg.params[2]*avgsen + reg.params[3]*np.square(avgsen) +\
reg.params[4]*ptime86 + reg.params[5]*qemp86 + reg.params[6]*inc86 + reg.params[7]*black + reg.params[8]*hispan

# Calculamos a expressão da derivada parcial de narr86 em relação à variável avgsen
exp = narr86.diff(avgsen)           

# Obtemos o ponto de reversão [o valor de 'avgsen' para o qual exp = 0]
ponto_reversao = sp.solveset(exp, symbol=avgsen, domain=sp.Reals)
ponto_reversao

{17.2768618691324}